In [1]:
from ola_RNN import * 
from ola_models import * 

import os, time, copy, math, re, json, pickle, random
import numpy as np
import pandas as pd

import torch, torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

from functools import partial 

cuda_available = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda_available else "cpu")
print(f'''using device {device}''')

path = !pwd
path = path[0]
print(path)

using device cuda:0
/home/r2/Documents/RNNexp


In [2]:
bs  = 20
sql = 2
lr  = 0.0005 

data          = pp_trumpdata(path+"/data/trump/", [0.9,0.95], bs)
data.train_dl = TweetDataLoader(data,data.train.tweets,bs,sql,shuffle=True)
data.valid_dl = TweetDataLoader(data,data.valid.tweets,bs,sql,shuffle=False)

In [3]:
n_stacks = 2
model  = cuda(SGRU(len(data.decoder), 150, n_stacks))
opt    = optim.RMSprop(model.parameters(), lr)
learn  = Learner(model, nn.NLLLoss(), opt, data, lr=lr)

cbs = CallbackHandler([CounterCallback(1000), StatsCallback()])

FYI number of iterations in a Twitter epoch is random, % progress will not be correctly displayed.


In [4]:
fit_rnn(1,learn,cbs)

getting validation
finished: 0.0%
getting validation
finished: 0.0%
getting validation
finished: 0.0%
getting validation
finished: 0.0%
getting validation
finished: 0.0%
getting validation
finished: 0.0%
getting validation
finished: 0.0%


KeyboardInterrupt: 

In [ ]:

def one_rnn_batch(xb,yb,cb):
    with torch.autograd.set_detect_anomaly(True):    
        pred, cb.learn.hidden, loss = cb.learn.model.batch_forward(xb,yb,cb.learn.hidden,cb.learn.loss_fn)
    if not cb.after_loss(loss): return   
    with torch.autograd.set_detect_anomaly(True):
        loss.backward()
    if not cb.after_backward(): return 
    cb.learn.opt.step()
    if not cb.after_step(): return
    cb.learn.opt.zero_grad()

def fit_rnn(epoches, learn, cb=None, itters=math.inf):
    hidden = learn.model.initHidden(learn.data.train_dl.bs)
    if not cb.begin_fit(learn):           return 
    for epoch in range(epoches):
        if not cb.begin_epoch(epoch):     return             
        for xb, yb in iter(learn.data.train_dl):   
            if not cb.begin_batch(xb,yb): return   
            one_rnn_batch(xb,yb,cb)
            if not cb.begin_validate():   return     
            if cb.do_stop():              break 
        if not cb.after_epoch():          return
    if not cb.after_fit():                return 
    return            

In [ ]:

#  is it the hidden layers 
#  or is it the input that has been changed in-place? 
#  Variable, is that different from a parameter in PyTorch? 

In [ ]:
model2  = cuda(GRU(len(data.decoder), 150))
opt2    = optim.RMSprop(model2.parameters(), lr)
learn2  = Learner(model2,  nn.NLLLoss(), opt2 , data, lr=lr)
fit_rnn(1,learn2,cbs)